In [ ]:

# Advanced Federated Learning for Privacy-Preserving Employee Well-being Monitoring
# Using Flower, PyTorch, and Opacus (with Energy-Efficient and DP Metrics)

# 1. Install dependencies
!pip install flwr opacus pandas scikit-learn torch matplotlib -q

# 2. Import Libraries
import flwr as fl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from opacus import PrivacyEngine
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import os

# 3. Load and Preprocess Data
from zipfile import ZipFile
with ZipFile("federated_clients_data.zip", 'r') as zip_ref:
    zip_ref.extractall("clients_data")

client_files = [f"clients_data/{fname}" for fname in os.listdir("clients_data") if fname.endswith(".csv")]

def preprocess(file_path):
    df = pd.read_csv(file_path)
    df = df.drop(columns=["User_ID", "Timestamp", "Device_ID"])
    df = pd.get_dummies(df, columns=["Activity"])
    X = df.drop("Stress_Level_1_10", axis=1).values
    y = (df["Stress_Level_1_10"] > 5).astype(int).values  # binary stress classification
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    return train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Model Definition
class StressNet(nn.Module):
    def __init__(self, input_dim):
        super(StressNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# 5. Flower Client with Differential Privacy and Metrics
class FLClient(fl.client.NumPyClient):
    def __init__(self, file_path):
        X_train, X_test, y_train, y_test = preprocess(file_path)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = StressNet(input_dim=X_train.shape[1]).to(self.device)
        self.loss_fn = nn.CrossEntropyLoss()

        X_train = torch.tensor(X_train, dtype=torch.float32)
        y_train = torch.tensor(y_train, dtype=torch.long)
        X_test = torch.tensor(X_test, dtype=torch.float32)
        y_test = torch.tensor(y_test, dtype=torch.long)

        self.trainloader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
        self.testloader = DataLoader(TensorDataset(X_test, y_test), batch_size=32)

        self.optimizer = optim.Adam(self.model.parameters(), lr=0.01)
        self.privacy_engine = PrivacyEngine()
        self.model, self.optimizer, self.trainloader = self.privacy_engine.make_private(
            module=self.model,
            optimizer=self.optimizer,
            data_loader=self.trainloader,
            noise_multiplier=1.0,
            max_grad_norm=1.0,
        )

    def get_parameters(self, config):
        return [val.cpu().numpy() for val in self.model.state_dict().values()]

    def set_parameters(self, parameters):
        state_dict = self.model.state_dict()
        for k, v in zip(state_dict.keys(), parameters):
            state_dict[k] = torch.tensor(v)
        self.model.load_state_dict(state_dict)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        self.model.train()
        for epoch in range(1):  # one local epoch
            for X_batch, y_batch in self.trainloader:
                X_batch, y_batch = X_batch.to(self.device), y_batch.to(self.device)
                self.optimizer.zero_grad()
                preds = self.model(X_batch)
                loss = self.loss_fn(preds, y_batch)
                loss.backward()
                self.optimizer.step()
        return self.get_parameters({}), len(self.trainloader.dataset), {
            "epsilon": self.privacy_engine.get_epsilon(delta=1e-5),
            "communication_kb": sum(p.numel() for p in self.model.parameters()) * 4 / 1024,
        }

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for X_batch, y_batch in self.testloader:
                X_batch, y_batch = X_batch.to(self.device), y_batch.to(self.device)
                outputs = self.model(X_batch)
                _, predicted = torch.max(outputs.data, 1)
                total += y_batch.size(0)
                correct += (predicted == y_batch).sum().item()
        acc = correct / total
        return float(1 - acc), total, {"accuracy": acc}

# 6. Start Federated Training Simulation
clients = [FLClient(fp) for fp in client_files]

def client_fn(cid):
    return clients[int(cid)]

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=len(clients),
    config=fl.server.ServerConfig(num_rounds=5),
    client_resources={"num_cpus": 1, "num_gpus": 0.5} if torch.cuda.is_available() else None,
)
